<a href="https://colab.research.google.com/github/michaeltreynolds/Fall22CS393/blob/main/embed/ConferenceScraperAndEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# You don't need to read all of this if you don't want to. Just "run all" if you want" and wait for it to download.

### Install Required Libraries

This cell installs the necessary Python libraries for web scraping, working with HTML content, and data manipulation.

- `requests`: Used to fetch HTML content from web pages.
- `beautifulsoup4`: Parses and extracts content from the HTML.
- `PyPDF2`: If you need to work with PDF files.


In [1]:
!pip install requests beautifulsoup4 PyPDF2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


### Web Scraping Conference Talks

This block contains helper functions for scraping LDS General Conference talks from the Church's website.

- `get_soup()`: Sends a request and parses the HTML from a given URL.
- `is_decade_page()`: Identifies if the URL is a decade selection page.
- `scrape_conference_pages()`: Fetches URLs for each conference (April/October) from the main page.
- `scrape_talk_urls()`: Retrieves URLs for individual talks from a specific conference.
- `scrape_talk_data()`: Scrapes the detailed data for each talk, such as title, speaker, calling, year, and season.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def get_soup(url):
    """Create a tree structure (BeautifulSoup) out of a GET request's HTML."""
    try:
        r = requests.get(url, allow_redirects=True)
        r.raise_for_status()
        print(f"Successfully fetched {r.url}")
        return BeautifulSoup(r.content, "html5lib")
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def is_decade_page(url):
    """Check if a page is a decade selection page."""
    return bool(re.search(r"/study/general-conference/\d{4}\d{4}", url))

def scrape_conference_pages(main_page_url):
    """Retrieve a list of URLs for each conference (year/month) from the main page."""
    soup = get_soup(main_page_url)
    if soup is None:
        print(f"Failed to fetch content from {main_page_url}")
        return []

    all_conference_links = []

    # Find all the links to individual conferences or decades
    links = [
        "https://www.churchofjesuschrist.org" + a["href"]
        for a in soup.find_all("a", href=True)
        if re.search(r"/study/general-conference/(\d{4}/(04|10)|\d{4}\d{4})", a["href"])
    ]

    for link in links:
        if is_decade_page(link):
            # Handle decade page
            decade_soup = get_soup(link)
            if decade_soup:
                year_links = [
                    "https://www.churchofjesuschrist.org" + a["href"]
                    for a in decade_soup.find_all("a", href=True)
                    if re.search(r"/study/general-conference/\d{4}/(04|10)", a["href"])
                ]
                all_conference_links.extend(year_links)
        else:
            all_conference_links.append(link)
            #print(f"direct: {link}")

    print(f"Total conference links found: {len(all_conference_links)}")
    print("Sample conference links:", all_conference_links[:5])
    print(all_conference_links)
    #https://www.churchofjesuschrist.org/study/general-conference/2024/10?lang=eng
    filtered_links = list(filter(lambda link: int(link.split("/")[5]) >= 2018, all_conference_links))
    #return all_conference_links
    return filtered_links

def scrape_talk_urls(conference_url):
    """Retrieve a list of URLs for each talk in a specific conference."""
    soup = get_soup(conference_url)
    if soup is None:
        return []

    talk_links = [
        "https://www.churchofjesuschrist.org" + a["href"]
        for a in soup.find_all("a", href=True)
        if re.search(r"/study/general-conference/\d{4}/(04|10)/.*", a["href"])
    ]

    # Remove duplicate links and session links
    talk_links = list(set(talk_links))
    talk_links = [link for link in talk_links if not link.endswith("session?lang=eng")]

    print(f"Found {len(talk_links)} talk links in {conference_url}")
    if talk_links:
        print("Sample talk links:", talk_links[:3])
    return talk_links

def scrape_talk_data(url):
    """Scrapes a single talk for data such as: title, conference, calling, speaker, content."""
    try:
        soup = get_soup(url)
        if soup is None:
            return {}

        title_tag = soup.find("h1")
        title = title_tag.text.strip() if title_tag else "No Title Found"

        conference_tag = soup.find("p", {"class": "subtitle"})
        conference = conference_tag.text.strip() if conference_tag else "No Conference Found"

        author_tag = soup.find("p", {"class": "author-name"})
        speaker = author_tag.text.strip() if author_tag else "No Speaker Found"

        calling_tag = soup.find("p", {"class": "author-role"})
        calling = calling_tag.text.strip() if calling_tag else "No Calling Found"

        content_array = soup.find("div", {"class": "body-block"})
        content = "\n\n".join(paragraph.text.strip() for paragraph in content_array.find_all("p")) if content_array else "No Content Found"

        footnotes = "\n".join(
            f"{idx}. {note.text.strip()}" for idx, note in enumerate(soup.find_all("li", {"class": "study-note"}), start=1)
        ) if soup.find_all("li", {"class": "study-note"}) else "No Footnotes Found"

        year = re.search(r'/(\d{4})/', url).group(1)
        season = "April" if "/04/" in url else "October"

        return {
            "title": title,
            "speaker": speaker,
            "calling": calling,
            "year": year,
            "season": season,
            "url": url,
            "talk": content,
            "footnotes": footnotes,
        }
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return {}

def scrape_talk_data_parallel(urls):
    """Scrapes all talks in parallel using ThreadPoolExecutor."""
    with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust `max_workers` as needed
        results = list(tqdm(executor.map(scrape_talk_data, urls), total=len(urls), desc="Scraping talks in parallel"))
    return [result for result in results if result]  # Filter out empty results

def main_scrape_process():
    main_url = "https://www.churchofjesuschrist.org/study/general-conference?lang=eng"
    conference_urls = scrape_conference_pages(main_url)
    print("Hello")
    print(conference_urls)
    all_talk_urls = []
    for conference_url in tqdm(conference_urls, desc="Scraping conferences"):
        all_talk_urls.extend(scrape_talk_urls(conference_url))

    print(f"Total talks found: {len(all_talk_urls)}")

    # Scrape talks in parallel
    conference_talks = scrape_talk_data_parallel(all_talk_urls)

    # Create DataFrame from the scraped data
    conference_df = pd.DataFrame(conference_talks)

    # Normalize Unicode and clean data
    for col in conference_df.columns:
        conference_df[col] = conference_df[col].apply(lambda x: unicodedata.normalize("NFD", x) if isinstance(x, str) else x)
        conference_df[col] = conference_df[col].apply(lambda x: x.replace("\t", "") if isinstance(x, str) else x)

    # Save to CSV and JSON
    conference_df.to_csv("conference_talks.csv", index=False)
    print("Scraping complete. Data saved to 'conference_talks.csv'.")

    conference_df.to_json("conference_talks.json", orient="records", indent=4)
    print("Data also saved to 'conference_talks.json'.")

# Run the scraper
start = time.time()
main_scrape_process()
end = time.time()
print(f"Total time taken: {end - start} seconds")
# main_url = "https://www.churchofjesuschrist.org/study/general-conference?lang=eng"
# conference_urls = scrape_conference_pages(main_url)

Successfully fetched https://www.churchofjesuschrist.org/study/general-conference?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/20102019?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/20002009?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/19901999?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/19801989?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/19711979?lang=eng
Total conference links found: 108
Sample conference links: ['https://www.churchofjesuschrist.org/study/general-conference/2024/10?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2024/04?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2023/10?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2023/04?lang=eng', 'https://www.churchofjesu

Scraping conferences:   0%|          | 0/14 [00:00<?, ?it/s]

Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/2024/10?lang=eng


Scraping conferences:   7%|▋         | 1/14 [00:00<00:10,  1.29it/s]

Found 36 talk links in https://www.churchofjesuschrist.org/study/general-conference/2024/10?lang=eng
Sample talk links: ['https://www.churchofjesuschrist.org/study/general-conference/2024/10/54cook?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2024/10/46wilcox?lang=eng', 'https://www.churchofjesuschrist.org/study/general-conference/2024/10/23villar?lang=eng']
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/2024/04?lang=eng


...

Scraping talks in parallel:  98%|█████████▊| 504/513 [00:59<00:00, 11.77it/s]

Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/2018/04/pure-love-the-true-sign-of-every-true-disciple-of-jesus-christ?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/2018/04/teaching-in-the-home-a-joyful-and-sacred-responsibility?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/2018/04/the-sustaining-of-church-officers?lang=engSuccessfully fetched https://www.churchofjesuschrist.org/study/general-conference/2018/04/let-us-all-press-on?lang=eng



Scraping talks in parallel: 100%|██████████| 513/513 [00:59<00:00,  8.59it/s]

Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/2018/04/saturday-morning-session?lang=eng
Successfully fetched https://www.churchofjesuschrist.org/study/general-conference/2018/04/prophets-speak-by-the-power-of-the-holy-spirit?lang=eng


Scraping complete. Data saved to 'conference_talks.csv'.
Data also saved to 'conference_talks.json'.
Total time taken: 75.74509763717651 seconds


In [3]:
from google.colab import files
files.download('conference_talks.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Clean Conference Talks Data

This code block performs several cleaning operations on the scraped data:
- Removes rows with "Church Auditing Department" in the calling.
- Removes unnecessary columns, like "conference" and "footnotes."
- Standardizes titles for speakers and callings (e.g., "Quorum of the 12" and "Seventy").
- Ensures uniformity across callings and speaker titles.
- Saves the cleaned data to a CSV file for easier analysis and sharing.

You can run this after scraping the data to ensure consistency and uniformity across the dataset.


In [ ]:
# prompt: I need to get embeddings for all the conference talks

!pip install sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer

def generate_embeddings(csv_file):
    """Generates and adds sentence embeddings to a CSV file of conference talks."""
    try:
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(csv_file)

        # Initialize the sentence transformer model
        model = SentenceTransformer('all-mpnet-base-v2')

        # Generate embeddings for the 'talk' column
        df['embeddings'] = df['talk'].apply(lambda x: model.encode(x).tolist())

        # Save the updated DataFrame to a new CSV file
        df.to_csv("conference_talks_with_embeddings.csv", index=False)
        print("Embeddings generated and saved to 'conference_talks_with_embeddings.csv'")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
generate_embeddings("conference_talks.csv")


In [ ]:
# prompt: given a search term find the the top 3 most similar conference talks from the embeddings

import pandas as pd
from sentence_transformers import SentenceTransformer, util

def find_similar_talks(search_term, top_k=3):
    """
    Finds the top_k most similar conference talks to a given search term based on embeddings.
    """
    try:
        df = pd.read_csv("conference_talks_with_embeddings.csv")

        # Initialize the sentence transformer model (ensure it's the same as used for embedding generation)
        model = SentenceTransformer('all-mpnet-base-v2')

        # Generate embedding for the search term
        search_embedding = model.encode(search_term)

        # Calculate cosine similarity between the search term embedding and talk embeddings
        similarities = []
        for index, row in df.iterrows():
            talk_embedding = row['embeddings']  # Assuming 'embeddings' column contains the embeddings
            if isinstance(talk_embedding, str): # Handle the case where embeddings are a string
              talk_embedding = eval(talk_embedding) # Convert back to a list of floats
            similarity = util.cos_sim(search_embedding, talk_embedding)
            similarities.append((index, similarity.item())) # Store index and the similarity score

        # Sort talks by similarity score in descending order
        similarities.sort(key=lambda x: x[1], reverse=True)

        # Get the top_k most similar talks
        top_talks = []
        for index, similarity in similarities[:top_k]:
            top_talks.append({
                "title": df.iloc[index]['title'],
                "speaker": df.iloc[index]['speaker'],
                "year": df.iloc[index]['year'],
                "similarity": similarity,
                "text": df.iloc[index]['talk']
            })

        return top_talks

    except FileNotFoundError:
        print("Error: 'conference_talks_with_embeddings.csv' not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []


# Example usage
#search_term = "building temples"  # Replace with your search term
search_term = "serious depression"  # Replace with your search term
similar_talks = find_similar_talks(search_term)

if similar_talks:
    print(f"Top 3 most similar talks to '{search_term}':")
    for talk in similar_talks:
        print(f"- Title: {talk['title']}, Speaker: {talk['speaker']}, Year: {talk['year']}, Similarity: {talk['similarity']:.4f}, {talk['text']}")
else:
    print("No similar talks found.")
